In [1]:
# Import packages
import osgeo
from os.path import join as pjoin
import pandas as pd
import numpy as np
import xarray as xr
import xrspatial as xrs
import rioxarray

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

import pystac
from pystac_client import Client
import pystac.item_collection as pyic
import planetary_computer
import requests
import stackstac

import rich.table
import dask.diagnostics

scratch_dir = './Project/scratch' 

from pystac_client import Client
import planetary_computer as pc

In [2]:
# Define your area of interest
aoi_EMV = { #aoi Everest and Makalu Valley Polygon
  "type": "Polygon",
  "coordinates": [
    [
      [86.63380152490207, 28.56507505979343],
      [87.74001280776696, 28.479331576213163],
      [87.7789385926358, 28.213296130511523],
      [87.61479871540345, 27.906713498710847],
      [87.61479871540345, 27.906713498710847],
      [87.04816717665705, 27.239072299890207],
      [87.04816717665705, 27.239072299890207],
      [28.54835734379745, 86.06605505495027],
      [86.63380152490207, 28.56507505979343]
    ]
  ]
}

In [3]:
aoi_KV = { # aoi Kachenjunga Valley
"type": "Polygon",
  "coordinates": [
    [
    [86.63380152490207, 28.56507505979343],
    [87.74001280776696, 28.479331576213163],
    [87.7789385926358, 28.213296130511523],
    [87.61479871540345, 27.906713498710847],
    [87.61479871540345, 27.906713498710847],
    [87.15652306810863, 27.36778735019498],
    [89.06841422478996, 27.269554114998044],
    [89.06841422478996, 28.56507505979343],
    [86.63380152490207, 28.56507505979343]
    ]
  ]
}

In [3]:
daterange = [
    {"interval": ["1985-09-01T00:00:00Z", "1985-11-01T00:00:00Z"]},
    {"interval": ["1990-09-01T00:00:00Z", "1990-11-01T00:00:00Z"]},
    {"interval": ["1995-09-01T00:00:00Z", "1995-11-01T00:00:00Z"]},
    {"interval": ["2000-09-01T00:00:00Z", "2000-11-01T00:00:00Z"]},
    {"interval": ["2005-09-01T00:00:00Z", "2005-11-01T00:00:00Z"]},
    {"interval": ["2010-09-01T00:00:00Z", "2010-11-01T00:00:00Z"]},
    {"interval": ["2015-09-01T00:00:00Z", "2015-11-01T00:00:00Z"]},
    {"interval": ["2020-09-01T00:00:00Z", "2020-11-01T00:00:00Z"]},
    {"interval": ["2022-09-01T00:00:00Z", "2022-11-01T00:00:00Z"]}]

def importdata(aoi, daterange):
    catalog = Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1"
        )
    # Define your search with CQL2 syntax
    search = catalog.search(filter_lang="cql2-json", filter={
        "op": "and",
        "args": [{"op": "s_intersects", "args": [{"property": "geometry"}, aoi_EMV]},
                {"op": "anyinteracts", "args": [{"property": "datetime"}, daterange[0]]},
                {"op": "=", "args": [{"property": "collection"}, "landsat-c2-l2"]},
                {"op": "<=", "args": [{"property": "eo:cloud_cover"}, 2]}
                ]
        }
    )


    first_item = next(search.get_items())
    pc.sign_item(first_item).assets

    charts = search.get_all_items()
    print('1885 items:',len(charts))
               
        #########################
    for t in daterange[1:]:
         # Search against the Planetary Computer STAC API
        catalog = Client.open(
            "https://planetarycomputer.microsoft.com/api/stac/v1")
        # Define your search with CQL2 syntax
        search = catalog.search(filter_lang="cql2-json", filter={
            "op": "and",
                "args": [
                    {"op": "s_intersects", "args": [{"property": "geometry"}, aoi_EMV]},
                    {"op": "anyinteracts", "args": [{"property": "datetime"}, t]},
                    {"op": "=", "args": [{"property": "collection"}, "landsat-c2-l2"]},
                    # data cant process more than k-amount of bands so cloud coverage can be set to almost 0
                    {"op": "<=", "args": [{"property": "eo:cloud_cover"}, 2]} 
                ]
            }  
        )

        first_item = next(search.get_items())
        pc.sign_item(first_item).assets

        items = search.get_all_items()
        print(t,'items: ',len(items))
        charts = charts+items

    print('Length total item set:',len(charts))     

charts = importdata(aoi_EMV, daterange)

1885 items: 7
{'interval': ['1990-09-01T00:00:00Z', '1990-11-01T00:00:00Z']} items:  5
{'interval': ['1995-09-01T00:00:00Z', '1995-11-01T00:00:00Z']} items:  22
{'interval': ['2000-09-01T00:00:00Z', '2000-11-01T00:00:00Z']} items:  40
{'interval': ['2005-09-01T00:00:00Z', '2005-11-01T00:00:00Z']} items:  65
{'interval': ['2010-09-01T00:00:00Z', '2010-11-01T00:00:00Z']} items:  107
{'interval': ['2015-09-01T00:00:00Z', '2015-11-01T00:00:00Z']} items:  123
{'interval': ['2020-09-01T00:00:00Z', '2020-11-01T00:00:00Z']} items:  199
{'interval': ['2022-09-01T00:00:00Z', '2022-11-01T00:00:00Z']} items:  94
Length total item set: 662


In [28]:
ds = stackstac.stack(planetary_computer.sign(charts), epsg=6207)
ds

<xarray.DataArray 'stackstac-cb917267e822b26400c75567ff49b9b7' (time: 662,
                                                                band: 22,
                                                                y: 183849,
                                                                x: 184368)>
dask.array<fetch_raster_window, shape=(662, 22, 183849, 184368), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/31)
  * time                         (time) datetime64[ns] 1985-09-06T07:01:14.40...
    id                           (time) <U31 'LT05_L2SP_169017_19850906_02_T2...
  * band                         (band) <U13 'qa' 'red' ... 'qa_aerosol'
  * x                            (x) float64 32.56 32.56 32.56 ... 90.02 90.02
  * y                            (y) float64 75.83 75.83 75.83 ... 26.35 26.35
    view:sun_azimuth             (time) float64 157.4 156.0 ... 131.5 158.5
    ...                           ...
    raster:bands                 (band) object {'unit': 'kelvin', 'scale': 0....
    classification:bitfields     (band) object None ... [{'name': 'fill', 'le...
    common_name                  (band) object None None None ... 'coastal' None
    center_wavelength            (band) object None None None ... 10.9 0.44 None
    full_width_half_max          (band) object None None None ... 0.59 0.02 None
    epsg                         int32 6207
Attributes:
    spec:           RasterSpec(epsg=6207, bounds=(32.558778242588346, 26.3457...
    crs:            epsg:6207
    transform:      | 0.00, 0.00, 32.56|\n| 0.00,-0.00, 75.83|\n| 0.00, 0.00,...
    resolution_xy:  (0.0003116537435516875, 0.0002691502682949931)

In [5]:
import rich.table

table = rich.table.Table("Asset Key", "Description")
for asset_key, asset in first_item.assets.items():
    table.add_row(asset_key, asset.title)

table

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Asset Key        ┃ Description                                                          ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ qa               │ Surface Temperature Quality Assessment Band                          │
│ ang              │ Angle Coefficients File                                              │
│ red              │ Red Band                                                             │
│ blue             │ Blue Band                                                            │
│ drad             │ Downwelled Radiance Band                                             │
│ emis             │ Emissivity Band                                                      │
│ emsd             │ Emissivity Standard Deviation Band                                   │
│ trad             │ Thermal Radiance Band                                                │
│ urad             │ Upwelled Radiance Band                                               │
│ atran            │ Atmospheric Transmittance Band                                       │
│ cdist            │ Cloud Distance Band                                                  │
│ green            │ Green Band                                                           │
│ nir08            │ Near Infrared Band 0.8                                               │
│ lwir11           │ Surface Temperature Band                                             │
│ swir16           │ Short-wave Infrared Band 1.6                                         │
│ swir22           │ Short-wave Infrared Band 2.2                                         │
│ coastal          │ Coastal/Aerosol Band                                                 │
│ mtl.txt          │ Product Metadata File (txt)                                          │
│ mtl.xml          │ Product Metadata File (xml)                                          │
│ mtl.json         │ Product Metadata File (json)                                         │
│ qa_pixel         │ Pixel Quality Assessment Band                                        │
│ qa_radsat        │ Radiometric Saturation and Terrain Occlusion Quality Assessment Band │
│ qa_aerosol       │ Aerosol Quality Assessment Band                                      │
│ tilejson         │ TileJSON with default rendering                                      │
│ rendered_preview │ Rendered preview                                                     │
└──────────────────┴──────────────────────────────────────────────────────────────────────┘

In [8]:
from IPython.display import Image

Image(url=first_item.assets["rendered_preview"].href, width=500)

In [98]:
with dask.diagnostics.ProgressBar():
    ts = ds.compute()

ts.rio.to_raster(pjoin(scratch_dir, 'ts.tif'), compress='LZW')

{'interval': ['1990-10-01T00:00:00Z', '1990-11-01T00:00:00Z']}